In [13]:
from IPython.display import display

import pylab as plt
%matplotlib inline

from sympy import *

from monom import *

In [14]:
t, x, y, tau, h, Re, p0 = symbols(r't, x, y, tau, h, Re, p_0', real=True)
u, v, p = (f(t, x, y) for f in symbols('u, v, p', cls=Function))

In [15]:
pda_f, pda_v = (u, v, p), (t, x, y)
pda_n, pda_clp = 5, 3
pda_p = Rational(0, 1), Rational(0, 1), Rational(0, 1)

In [16]:
Monom.variables = 1 + len(pda_v)
Monom.cmp = Monom.TOPdeglex
Monom.zero = Monom(0 for v in range(Monom.variables))
pda_fun = dict(zip(pda_f,\
             (Monom(0 if v else i for v in range(Monom.variables))\
              for i in range(1, 1 + len(pda_f)))))
pda_var = dict(zip(pda_v,\
             (Monom(0 if v != i else 1 for v in range(Monom.variables))\
              for i in range(1, Monom.variables))))

In [17]:
def T(f, i1, j1, k1):
    return sum(sum(sum(\
        diff(f, t, i, x, j, y, k)*(tau*(i1+pda_p[0]))**i \
                                   *(h*(j1+pda_p[1]))**j \
                                   *(h*(k1+pda_p[2]))**k/\
                 (factorial(i)*factorial(j)*factorial(k))\
        for i in range(pda_n-j-k))\
        for k in range(pda_n-j))  \
        for j in range(pda_n))

In [18]:
expand((T(u, 0, 0, 1)-T(u, 0, 0, -1))/(2*h))

h**2*Derivative(u(t, x, y), (y, 3))/6 + Derivative(u(t, x, y), y)

In [19]:
def clip(f):
    f = f.expand()
    return [f.coeff(tau, 0).coeff(h, 0),\
            f.coeff(h, 0).coeff(tau, 1),\
            f.coeff(tau, 0).coeff(h, 2)]

def df2m(a):
    assert a.func == Derivative
    m = pda_fun[a.args[0]]
    for xi in a.args[1:]:
        if isinstance(xi, Symbol):
            m = m*pda_var[xi]
        else:
            m = m*pda_var[xi[0]]**xi[1]
    return m

def m2df(m):
    r = pda_f[m[0]-1]
    for i in range(1, len(m)):
        r = r.diff(pda_v[i-1], m[i])
    return r

def findDiv(a, d):
    r = None
    def find(a, r):
        if a.args:
            if a.func == Derivative and a.args[0] in pda_fun:
                m = df2m(a)
                if m.divisible(d) and (not r or m.cmp(r) > 0):
                    r = m
            else:
                for s in a.args:
                    r = find(s, r)
        return r
    return find(a, r)

def reduction(f1, f2, m, c, shift):
    assert shift < pda_clp
    r = [f1[i] for i in range(shift)]
    if not m:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift]*c))
    else:
        for i in range(shift, pda_clp):
            r.append(expand(f1[i] - f2[i-shift].diff(*m)*c))
    return r

def NF(f, df, G, head=False, trace=False):
    assert len(df) == len(G)
#     print(df2m(df[0]))
    ms = [df2m(d) for d in df]
    for i in range(0 if head else 1, pda_clp):
        t = 0
        if f[i]:
            while True:
                r = None
                for l in range(len(ms)):
                    r = findDiv(f[i], ms[l])
                    if r:
                        break
                if not r: 
                    break
                c, deg, m = 0, 7, m2df(r)
                while c == 0:
                    c = f[i].coeff(m, deg)
                    deg -= 1
                    assert deg >= 0
#                 print(c, m, deg+1)
                if deg:
                    c *= m**deg
                m = r/ms[l]
                d = []
                for k in range(len(pda_v)):
                    if m[k+1] > 0:
                        d.append(pda_v[k])
                        if m[k+1] > 1:
                            d.append(m[k+1])
                if trace:
                    print(">"*12)
                    eq = Symbol("eq%d" % (l+1), real=True)
                    if d:
                        display(Derivative(*tuple([eq] + d))*c*h**i)
                    else:
                        display(eq*c*h**i)
                f = reduction(f, G[l], tuple(d), c/G[l][0].coeff(df[l]), i)
                if trace:
                    print("res =")
                    display(f)
                    print("<"*12)
                t += 1
#                 if t > 6: break
    return f

def compact(f):
    def cmpct(a):
        if not a.args:
            return a
        else:
            if a in pda_fun:
                return Symbol("%s" % a.func, real=True)
            elif a.func == Derivative and a.args[0] in pda_fun:
                m = []
                for xi in a.args[1:]:
                    if isinstance(xi, Symbol):
                        m.append(str(xi))
                    else:
                        m.append(str(xi[0])*xi[1])
                return Symbol("%s_{%s}" % (a.args[0].func, "".join(m)), real=True)
            else:
                return a.func(*tuple(cmpct(s) for s in a.args))
    return cmpct(f)

def prn(a):
    display(compact(a[0]))
    print("tau =>")
    display(compact(a[1]))
    print("h^2 =>")
    display(compact(a[2]))
    
def prnlatex(a):
    print(latex(compact(a[0])))
    print(r"+\tau\big(")
    print(latex(compact(a[1]))) 
    print(r"\big)")
    print(r"+h^2\big(")
    print(latex(compact(a[2]))) 
    print(r"\big)")    

In [34]:
def Dt(a):
    return (T(a, 1, 0, 0) - T(a, 0, 0, 0))/(tau)
def It(a):
    return (T(a, 1, 0, 0) + T(a, 0, 0, 0))/2
def Ut(a):
    return T(a, 1, 0, 0)
def Dx(a):
    return (T(a, 0, 1, 0) - T(a, 0, -1, 0))/(2*h)
def Dy(a):
    return (T(a, 0, 0, 1) - T(a, 0, 0, -1))/(2*h)

def DD(a):
    return (T(a, 0, 1, 0) + T(a, 0, 0, 1) +\
                -4*T(a, 0, 0, 0) +\
           T(a, 0, -1, 0) + T(a, 0, 0, -1))/h**2

def DD2(a):
    return (T(a, 0, 2, 0) + T(a, 0, 0, 2) +\
                -4*T(a, 0, 0, 0) +\
           T(a, 0, -2, 0) + T(a, 0, 0, -2))/(4*h**2)

def Dxx(a):
    return (T(a, 0, 2, 0) - 2*T(a, 0, 0, 0) + T(a, 0, -2, 0))/(4*h**2)
def Dxy(a):
    return (T(a, 0, 1, 1) - T(a, 0, 1, -1) -\
            T(a, 0, -1, 1) + T(a, 0, -1, -1))/(4*h**2)
def Dyy(a):
    return (T(a, 0, 0, 2) - 2*T(a, 0, 0, 0) + T(a, 0, 0, -2))/(4*h**2)

def DDx(a):
    return ((T(a, 0, 2, 0) + T(a, 0, 1, 1) -\
             4*T(a, 0, 1, 0) + T(a, 0, 1, -1)) -\
            (T(a, 0, -1, 1) - 4*T(a, 0, -1, 0) +\
             T(a, 0, -2, 0) + T(a, 0, -1, -1)))/(2*h**3)
def DDy(a):
    return ((T(a, 0, 1, 1) + T(a, 0, 0, 2) -\
             4*T(a, 0, 0, 1) + T(a, 0, -1, 1)) -\
            (T(a, 0, 1, -1) - 4*T(a, 0, 0, -1) +\
             T(a, 0, -1, -1) + T(a, 0, 0, -2)))/(2*h**3)

In [21]:
def build(F1, F2, F3, F4):
    prn(F1);print("-"*25)
    f1 = NF(F1, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [F1, F2, F3, F4], head=False)
    prn(f1)
    prnlatex(f1)
    td = [f.subs(TD).doit().expand().simplify() for f in f1]
    prn(td)
    prnlatex(td)
    kf = [f.subs(KF).doit().expand().simplify() for f in f1]
    prn(kf)
    prnlatex(kf)

    prn(F2);print("-"*25)
    f2 = NF(F2, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, F2, F3, F4], head=False)
    prn(f2)
    prnlatex(f2)
    td = [f.subs(TD).doit().expand().simplify() for f in f2]
    prn(td)
    prnlatex(td)
    kf = [f.subs(KF).doit().expand().simplify() for f in f2]
    prn(kf)
    prnlatex(kf)

    prn(F3);print("-"*25)
    f3 = NF(F3, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, F3, F4], head=False)
    prn(f3)
    prnlatex(f3)
    td = [f.subs(TD).doit().expand().simplify() for f in f3]
    prn(td)
    prnlatex(td)
    kf = [f.subs(KF).doit().expand().simplify() for f in f3]
    prn(kf)
    prnlatex(kf)

    prn(F4);print("-"*25)
    f4 = NF(F4, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, F4], head=False)
    prn(f4)
    prnlatex(f4)
    td = [f.subs(TD).doit().expand().simplify() for f in f4]
    prn(td)
    prnlatex(td)
    kf = [f.subs(KF).doit().expand().simplify() for f in f4]
    prn(kf)
    prnlatex(kf)

    spoly = [a.diff(y, 2) - b.diff(y) for a, b in zip(f2, f3)]
    spoly = NF(spoly, [u.diff(x), u.diff(y, 2), v.diff(x, 2), p.diff(x, 2)], [f1, f2, f3, f4], head=True)
    prn(spoly)
    prnlatex(spoly)

**Taylor decaying**

This is a classical Navier-Stokes problem, in general used to state the con-
vergence order of the considered formulae. Exact solution is
\begin{equation} \label{eq1}
\left\lbrace 
\begin{array}{l}
 u = -e^{-2t/\mathrm{Re}} \cos(x) \sin(y) ,\\
 v =  e^{-2t/\mathrm{Re}} \sin(x) \cos(y) ,\\
 p = -e^{-4t/\mathrm{Re}} (\cos(2x) + \cos(2y))/4 ,
\end{array}
\right. 
\end{equation} 

In [22]:
TD = {u: -exp(-2*t/Re)*cos(x)*sin(y),\
      v:  exp(-2*t/Re)*sin(x)*cos(y),\
      p: -exp(-4*t/Re)*(cos(2*x)+cos(2*y))/4}

**Kovasznay flow**

This is a classical Navier-Stokes problem, in general used to state the con-
vergence order of the considered formulae. Exact solution is
\begin{equation} \label{eq2}
\left\lbrace 
\begin{array}{l}
 u = 1-e^{\lambda x} \cos(2 \pi y) ,\\
 v = \lambda/{2\pi} e^{\lambda x} \sin(2 \pi y) ,\\
 p = p_0-\frac{1}{2}e^{2\lambda x} ,
\end{array}
\right. 
\end{equation}
where $\lambda = \mathrm{Re}/2-\sqrt{\mathrm{Re}^2/4+4\pi^2}$.

In [23]:
Lambda = Re/2 - sqrt(Re**2/4 + 4*pi**2)
KF = {u: 1 - exp(Lambda*x)*cos(2*pi*y),\
      v: Lambda/(2*pi)*exp(Lambda*x)*sin(2*pi*y),\
      p: p0 - exp(2*Lambda*x)/2}

\begin{equation}\label{4}
\left\lbrace 
\begin{array}{rl}
F^1:& D_{1x}u^{n}_{j+l+m,\,k+l} +D_{1y}v^{n}_{j+l+m,\,k+l}=0 ,\\[4pt]
F^2:& \dfrac{u^{n+1}_{j+l,\,k+l}-u^{n}_{j+l,\,k+l}}{\tau} + I_t\left( u^{n}_{j+l,\,k+l}D_{2x}u^{n}_{j+l,\,k+l} + {}\right. \\[4pt]
&\left. \qquad {}+v^{n}_{j+l,\,k+l}D_{2y}u^{n}_{j+l,\,k+l}+D_{3x}p^{n}_{j+l,\,k+l}-\dfrac{1}{\mathrm{Re}}\bigtriangleup u^{n}_{j+l,\,k+l}\right) =0 ,\\[4pt]
F^3:& \dfrac{v^{n+1}_{j+l,\,k+l}-v^{n}_{j+l,\,k+l}}{\tau} + I_t\left( u^{n}_{j+l,\,k+l}D_{2x}v^{n}_{j+l,\,k+l} + {}\right. \\[4pt]
&\left. \qquad {}+v^{n}_{j+l,\,k+l}D_{2y}v^{n}_{j+l,\,k+l}+D_{3y}p^{n}_{j+l,\,k+l}-\dfrac{1}{\mathrm{Re}}\bigtriangleup v^{n}_{j+l,\,k+l}\right) =0 ,\\[4pt]
 F^4:& D_{1x}\left(u^{n}_{j+l,\,k+l}D_{2x}u^{n}_{j+l,\,k+l}+v^{n}_{j+l,\,k+l}D_{2y}u^{n}_{j+l,\,k+l}\right) + {} \qquad \\[4pt]
    & \qquad {} +D_{1y}\left(u^{n}_{j+l,\,k+l}D_{2x}v^{n}_{j+l,\,k+l}+v^{n}_{j+l,\,k+l}D_{2y}v^{n}_{j+l,\,k+l}\right) + {} \\[4pt]
    & \hfill {}  +\left( D_{1x}D_{3x}p^{n}_{j+l,\,k+l} + D_{1y}D_{3y}p^{n}_{j+l,\,k+l}\right)  =0 .
\end{array}
\right. 
\end{equation}

In [24]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + u*Dx(u) + v*Dy(u) + Dx(p) - DD(u)/Re)
F3 = clip(Dt(v) + u*Dx(v) + v*Dy(v) + Dy(p) - DD(v)/Re)
F4 = clip(Dx(u*Dx(u) + v*Dy(u) + Dx(p)) + Dy(u*Dx(v) + v*Dy(v) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [25]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + u*Dx(u) + v*Dy(u) + Dx(p) - DD2(u)/Re)
F3 = clip(Dt(v) + u*Dx(v) + v*Dy(v) + Dy(p) - DD2(v)/Re)
F4 = clip(Dx(u*Dx(u) + v*Dy(u) + Dx(p)) + Dy(u*Dx(v) + v*Dy(v) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [26]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + It(u*Dx(u) + v*Dy(u) + Dx(p) - DD(u)/Re))
F3 = clip(Dt(v) + It(u*Dx(v) + v*Dy(v) + Dy(p) - DD(v)/Re))
F4 = clip(Dx(u*Dx(u) + v*Dy(u) + Dx(p)) + Dy(u*Dx(v) + v*Dy(v) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [27]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + It(u*Dx(u) + v*Dy(u) + Dx(p) - DD2(u)/Re))
F3 = clip(Dt(v) + It(u*Dx(v) + v*Dy(v) + Dy(p) - DD2(v)/Re))
F4 = clip(Dx(u*Dx(u) + v*Dy(u) + Dx(p)) + Dy(u*Dx(v) + v*Dy(v) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [28]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + Dx(u**2) + Dy(v*u) + Dx(p) - DD(u)/Re)
F3 = clip(Dt(v) + Dx(u*v) + Dy(v**2) + Dy(p) - DD(v)/Re)
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [29]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + Dx(u**2) + Dy(v*u) + Dx(p) - DD2(u)/Re)
F3 = clip(Dt(v) + Dx(u*v) + Dy(v**2) + Dy(p) - DD2(v)/Re)
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [30]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + It(u*Dx(u) + v*Dy(u) + Dx(p) - DD(u)/Re))
F3 = clip(Dt(v) + It(u*Dx(v) + v*Dy(v) + Dy(p) - DD(v)/Re))
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [31]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + It(u*Dx(u) + v*Dy(u) + Dx(p) - DD2(u)/Re))
F3 = clip(Dt(v) + It(u*Dx(v) + v*Dy(v) + Dy(p) - DD2(v)/Re))
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [32]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + It(Dx(u**2) + Dy(v*u) + Dx(p) - DD(u)/Re))
F3 = clip(Dt(v) + It(Dx(u*v) + Dy(v**2) + Dy(p) - DD(v)/Re))
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [33]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + It(Dx(u**2) + Dy(v*u) + Dx(p) - DD2(u)/Re))
F3 = clip(Dt(v) + It(Dx(u*v) + Dy(v**2) + Dy(p) - DD2(v)/Re))
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [35]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + Ut(u*Dx(u) + v*Dy(u) + Dx(p) - DD(u)/Re))
F3 = clip(Dt(v) + Ut(u*Dx(v) + v*Dy(v) + Dy(p) - DD(v)/Re))
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [36]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + Ut(u*Dx(u) + v*Dy(u) + Dx(p) - DD2(u)/Re))
F3 = clip(Dt(v) + Ut(u*Dx(v) + v*Dy(v) + Dy(p) - DD2(v)/Re))
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [37]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + Ut(Dx(u**2) + Dy(v*u) + Dx(p) - DD(u)/Re))
F3 = clip(Dt(v) + Ut(Dx(u*v) + Dy(v**2) + Dy(p) - DD(v)/Re))
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


In [38]:
F1 = clip(Dx(u) + Dy(v))
F2 = clip(Dt(u) + Ut(Dx(u**2) + Dy(v*u) + Dx(p) - DD2(u)/Re))
F3 = clip(Dt(v) + Ut(Dx(u*v) + Dy(v**2) + Dy(p) - DD2(v)/Re))
F4 = clip(Dx(Dx(u**2) + Dy(v*u) + Dx(p)) + Dy(Dx(u*v) + Dy(v**2) + Dy(p)))

build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


0

h^2 =>


u_{xxx}/6 + v_{yyy}/6

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)


## Метод MAC ##
<img src="./F17.13.png"/>
<img src="./F17.8.png"/>
<img src="./F17.10.png"/>

In [41]:
p12 = Rational(1, 2)
def F(j):
    return T(u, 0, j, 0) + tau/h*(\
        (T(u, 0, j+1, 0) - 2*T(u, 0, j, 0) + T(u, 0, j-1, 0))/(Re*h) +\
        (T(u, 0, j, 1) -   2*T(u, 0, j, 0) + T(u, 0, j, -1))/(Re*h) -\
        ((T(u, 0, j+1, 0) + T(u, 0, j, 0))**2 - (T(u, 0, j, 0) + T(u, 0, j-1, 0))**2)/4 -\
        ((T(u, 0, j, 1) + T(u, 0, j, 0)) *(T(v, 0, j+1, p12)  + T(v, 0, j-1, p12)) -\
         (T(u, 0, j, 0) + T(u, 0, j, -1))*(T(v, 0, j+1, -p12) + T(v, 0, j-1, -p12)))/4)

def G(k):
    return T(v, 0, 0, k) + tau/h*(\
        (T(v, 0, 1, k) -   2*T(v, 0, 0, k) + T(v, 0, -1, k))/(Re*h) +\
        (T(v, 0, 0, k+1) - 2*T(v, 0, 0, k) + T(v, 0, 0, k-1))/(Re*h) -\
        ((T(u, 0, p12, k+1)  + T(u, 0, p12, k-1))* (T(v, 0, 1, k) + T(v, 0, 0, k)) -\
         (T(u, 0, -p12, k+1) + T(u, 0, -p12, k-1))*(T(v, 0, 0, k) + T(v, 0, -1, k)))/4 -\
        ((T(v, 0, 0, k+1) + T(v, 0, 0, k))**2 - (T(v, 0, 0, k) + T(v, 0, 0, k-1))**2)/4)

F1 = clip(((T(u, 0, p12, 0) - T(u, 1, -p12, 0)) +\
           (T(v, 0, 0, p12) - T(v, 1, 0, -p12)))/h)
F2 = clip((T(u, 1, p12, 0) - F(p12))/tau +\
          (T(p, 1, 1, 0) - T(p, 1, 0, 0))/h)
F3 = clip((T(v, 1, 0, p12) - G(p12))/tau +\
          (T(p, 1, 0, 1) - T(p, 1, 0, 0))/h)
F4 = clip((T(p, 1, 1, 0) - 2*T(p, 1, 0, 0) + T(p, 1, -1, 0))/h**2 +\
          (T(p, 1, 0, 1) - 2*T(p, 1, 0, 0) + T(p, 1, 0, -1))/h**2 -\
         ((F(p12) - F(-p12)) + (G(p12) - G(-p12)))/(tau*h))
build(F1, F2, F3, F4)

u_{x} + v_{y}

tau =>


u_{tx}/2 + v_{ty}/2

h^2 =>


u_{xxx}/24 + v_{yyy}/24

...

tau =>


0

h^2 =>


0

0
+\tau\big(
0
\big)
+h^2\big(
0
\big)
